In [2]:
ls!pip install git+https://github.com/tencent-ailab/IP-Adapter.git
!pip install insightface
!pip install face_alignment
!pip install onnxruntime
!pip install onnxruntime-gpu


In [3]:
import os
import cv2
import torch
from insightface.app import FaceAnalysis
from diffusers import StableDiffusionPipeline, DDIMScheduler, AutoencoderKL
from PIL import Image
import numpy as np
from ip_adapter.ip_adapter_faceid import IPAdapterFaceID


C:\Users\jet\AppData\Roaming\Python\Python312\site-packages\albumentations\__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.6' (you have '2.0.5'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [4]:
def create_test_image():
    img = np.zeros((512, 512, 3), dtype=np.uint8)
    cv2.circle(img, (256, 256), 100, (255, 255, 255), -1)
    cv2.circle(img, (216, 216), 20, (0, 0, 0), -1)
    cv2.circle(img, (296, 216), 20, (0, 0, 0), -1)
    cv2.ellipse(img, (256, 296), (50, 30), 0, 0, 180, (0, 0, 0), 2)
    cv2.imwrite("test_face.jpg", img)
    return "test_face.jpg"


def extract_face_embedding(image_path):
    app = FaceAnalysis(name="buffalo_l", providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])
    app.prepare(ctx_id=0, det_size=(640, 640))
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Could not read image from {image_path}")
    faces = app.get(image)
    if not faces:
        raise ValueError("No face detected in the image")
    return torch.from_numpy(faces[0].normed_embedding).unsqueeze(0)


def generate_images(prompt, face_image_path, num_samples=4):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    # Scheduler and models
    scheduler = DDIMScheduler(
        num_train_timesteps=1000,
        beta_start=0.00085,
        beta_end=0.012,
        beta_schedule="scaled_linear",
        clip_sample=False,
        set_alpha_to_one=False,
        steps_offset=1,
    )
    vae = AutoencoderKL.from_pretrained("stabilityai/sd-vae-ft-mse").to(dtype=torch.float16)
    pipe = StableDiffusionPipeline.from_pretrained(
        "SG161222/Realistic_Vision_V4.0_noVAE",
        torch_dtype=torch.float16,
        scheduler=scheduler,
        vae=vae,
        feature_extractor=None,
        safety_checker=None
    )
    ip_model = IPAdapterFaceID(pipe, "ip-adapter-faceid_sd15.bin", device)

    faceid_embeds = extract_face_embedding(face_image_path)
    negative_prompt = (
        "old, wrinkles, aged, elderly, man, masculine, bad anatomy, worst quality,"
        " low quality, blurry"
    )
    images = ip_model.generate(
        prompt=prompt,
        negative_prompt=negative_prompt,
        faceid_embeds=faceid_embeds,
        num_samples=num_samples,
        width=512,
        height=768,
        num_inference_steps=40,
        seed=2023
    )
    return images

In [5]:
face_image_path = "IMG_6909.jpg"

# List of prompts to test
prompts = [
    "a photo of a young woman in a garden, wearing a red dress",
    "a portrait of a woman wearing a vintage hat, sitting by the sea at sunset",
    "a stylized cinematic headshot of a smiling woman with freckles outdoors",
    "a high-resolution photograph of a woman walking through a sunflower field",
    "a photo of a woman on a modern city street at night with neon lights",
    "a renaissance painting style portrait of a woman in a forest",
    "a whimsical fairy-like woman sitting on a mushroom",
    "a photorealistic image of a woman wading through a misty mountain stream",
    "a close-up portrait of a woman with dramatic lighting, film noir style",
    "a vibrant fashion photo of a woman in a floral dress on a beach"
]

# Base output directory
output_base = "generated_images"
os.makedirs(output_base, exist_ok=True)

for idx, prompt in enumerate(prompts, start=1):
    # Create subfolder for each prompt
    folder_name = f"prompt_{idx:02d}"
    output_dir = os.path.join(output_base, folder_name)
    os.makedirs(output_dir, exist_ok=True)

    print(f"Generating images for prompt #{idx}: {prompt}")
    try:
        images = generate_images(prompt, face_image_path)
        for i, img in enumerate(images):
            img_path = os.path.join(output_dir, f"{idx}_{i+1}.png")
            img.save(img_path)
        print(f"Saved {len(images)} images in {output_dir}")
    except Exception as e:
        print(f"Error with prompt #{idx}: {e}")


Generating images for prompt #1: a photo of a young woman in a garden, wearing a red dress


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

C:\Users\jet\AppData\Local\anaconda3\envs\newPy\Lib\site-packages\diffusers\models\lora.py:208: FutureWarning: `LoRALinearLayer` is deprecated and will be removed in version 1.0.0. Use of `LoRALinearLayer` is deprecated. Please switch to PEFT backend by installing PEFT: `pip install peft`.
  deprecate("LoRALinearLayer", "1.0.0", deprecation_message)


Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CUDAExecutionProvider': {'device_id': '0', 'has_user_compute_stream': '0', 'cudnn_conv1d_pad_to_nc1d': '0', 'user_compute_stream': '0', 'gpu_external_alloc': '0', 'gpu_mem_limit': '18446744073709551615', 'enable_cuda_graph': '0', 'gpu_external_free': '0', 'gpu_external_empty_cache': '0', 'arena_extend_strategy': 'kNextPowerOfTwo', 'cudnn_conv_algo_search': 'EXHAUSTIVE', 'do_copy_in_default_stream': '1', 'cudnn_conv_use_max_workspace': '1', 'tunable_op_enable': '0', 'tunable_op_tuning_enable': '0', 'tunable_op_max_tuning_duration_ms': '0', 'enable_skip_layer_norm_strict_mode': '0', 'prefer_nhwc': '0', 'use_ep_level_unified_stream': '0', 'use_tf32': '1', 'sdpa_kernel': '0', 'fuse_conv_bias': '0'}, 'CPUExecutionProvider': {}}
find model: C:\Users\jet/.insightface\models\buffalo_l\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'],

  0%|          | 0/40 [00:00<?, ?it/s]

...

Saved 4 images in generated_images\prompt_09
Generating images for prompt #10: a vibrant fashion photo of a woman in a floral dress on a beach


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CUDAExecutionProvider': {'device_id': '0', 'has_user_compute_stream': '0', 'cudnn_conv1d_pad_to_nc1d': '0', 'user_compute_stream': '0', 'gpu_external_alloc': '0', 'gpu_mem_limit': '18446744073709551615', 'enable_cuda_graph': '0', 'gpu_external_free': '0', 'gpu_external_empty_cache': '0', 'arena_extend_strategy': 'kNextPowerOfTwo', 'cudnn_conv_algo_search': 'EXHAUSTIVE', 'do_copy_in_default_stream': '1', 'cudnn_conv_use_max_workspace': '1', 'tunable_op_enable': '0', 'tunable_op_tuning_enable': '0', 'tunable_op_max_tuning_duration_ms': '0', 'enable_skip_layer_norm_strict_mode': '0', 'prefer_nhwc': '0', 'use_ep_level_unified_stream': '0', 'use_tf32': '1', 'sdpa_kernel': '0', 'fuse_conv_bias': '0'}, 'CPUExecutionProvider': {}}
find model: C:\Users\jet/.insightface\models\buffalo_l\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'],

  0%|          | 0/40 [00:00<?, ?it/s]

Saved 4 images in generated_images\prompt_10
